**Lets have fun!**

In [ ]:
# Python 3 Environment
# Kaggle/Python docker image: https://github.com/kaggle/docker-python

# IMPORTS DUH!
import os
import csv
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cv2

# cool ones
from keras.models import Sequential
from keras.layers import Convolution2D, MaxPooling2D, Dense, Dropout, Flatten, Activation

#from keras.optimizers import RMSprop, SGD
#from keras.callbacks import ModelCheckpoint, Callback, EarlyStopping
#from keras.utils import np_utils
#from keras.regularizers import l2
#from keras.utils.np_utils import to_categorical

from keras.utils import np_utils
from keras import backend as K
K.set_image_dim_ordering('th')

# set a random seed
np.random.seed(1)

print("let's go!")

In [ ]:
# set the training path
train_path = "../input/train"
test_path = "../input/test"
base_path = "."

print(train_path)
print(test_path)

In [ ]:
# load filenames
images_dog = [img for img in os.listdir(train_path) if "dog" in img] # creates a list of all the dog filenames
images_cat = [img for img in os.listdir(train_path) if "cat" in img] # creates a list of all the dog filenames
images_test = [img for img in os.listdir(test_path)] # creates a list of all the file names

train_dog = images_dog[:100] # select sample of dog filenames
train_cat = images_cat[:100] # select sample of cat filenames
valid_dog = images_dog[101:201]
valid_cat = images_cat[101:201]

train_list = train_dog + train_cat # join sample filenames
valid_list = valid_dog + valid_cat # join sample filenames
test_list = images_test[:100]

print("train list: ", train_list[:10])
print()
print("valid list: ", valid_list[:10])
print()
print("test list: ", test_list[:10])

In [ ]:
# set image variables for when loading
img_size_row = 224
img_size_col = 224

print("row:", img_size_row, "col:", img_size_col)

In [ ]:
# create arrays for training and labels
train = np.ndarray(shape=(len(train_list), img_size_row, img_size_col))
train_labels = np.ndarray(len(train_list))

valid = np.ndarray(shape=(len(valid_list), img_size_row, img_size_col))
valid_labels = np.ndarray(len(valid_list))

test = np.ndarray(shape=(len(test_list), img_size_row, img_size_col))

print("train array shape: ", train.shape)
print("train_labels array shape: ", train_labels.shape)
print()
print("valid array shape: ", valid.shape)
print("valid_labels array shape: ", valid_labels.shape)
print()
print("test array shape: ", test.shape)

**PreProcess**

In [ ]:
#### for loop to bring in the images and proces

##################################            
#### train #######################
##################################
for i, img_path in enumerate(train_list): # only for images in the train list

        # read in image
        img = cv2.imread(os.path.join(train_path, img_path), 0) # the 0 indicates grayscale!

        # create a canvas for the image for cropping and resziing while maintain image shape
        if img.shape[0] < img.shape[1]:
            img.resize([np.max(img.shape), np.max(img.shape)])

        elif img.shape[0] > img.shape[1]:
            img = np.array(img, order ='F')
            img.resize([np.max(img.shape), np.max(img.shape)])

        else:
            img
            
        # resize each image
        img = cv2.resize(img, (img_size_row, img_size_col), interpolation=cv2.INTER_CUBIC)

        #scale between 0 and 1
        img = (img-np.min(img))/(np.max(img)-np.min(img))

        # load the img into train
        train[i] = img

        # for each img in train label 1 if dog else 0 (cat)
        if "dog" in img_path:
            train_labels[i] = 1
        else:
            train_labels[i] = 0

            
##################################            
#### valid #######################
##################################

for i, img_path in enumerate(valid_list): # only for images in the train list

        # read in image (valid list also come from train path)
        img = cv2.imread(os.path.join(train_path, img_path), 0) # the 0 indicates grayscale!

        # create a canvas for the image for cropping and resziing while maintain image shape
        if img.shape[0] < img.shape[1]:
            img.resize([np.max(img.shape), np.max(img.shape)])

        elif img.shape[0] > img.shape[1]:
            img = np.array(img, order ='F')
            img.resize([np.max(img.shape), np.max(img.shape)])

        else:
            img
            
        # resize each image
        img = cv2.resize(img, (img_size_row, img_size_col), interpolation=cv2.INTER_CUBIC)

        #scale between 0 and 1
        img = (img-np.min(img))/(np.max(img)-np.min(img))

        # load the img into valid
        valid[i] = img

        # for each img in train label 1 if dog else 0 (cat)
        if "dog" in img_path:
            valid_labels[i] = 1
        else:
            valid_labels[i] = 0
            
##################################            
#### test ########################
##################################

for i, img_path in enumerate(test_list): # only for images in the train list

        # read in image
        img = cv2.imread(os.path.join(test_path, img_path), 0) # the 0 indicates grayscale!

        # create a canvas for the image for cropping and resziing while maintain image shape
        if img.shape[0] < img.shape[1]:
            img.resize([np.max(img.shape), np.max(img.shape)])

        elif img.shape[0] > img.shape[1]:
            img = np.array(img, order ='F')
            img.resize([np.max(img.shape), np.max(img.shape)])

        else:
            img
            
        # resize each image
        img = cv2.resize(img, (img_size_row, img_size_col), interpolation=cv2.INTER_CUBIC)

        #scale between 0 and 1
        img = (img-np.min(img))/(np.max(img)-np.min(img))

        # load the img into valid
        test[i] = img

In [ ]:
#show processed image train
n = 50
train_check = cv2.imread(os.path.join(train_path, train_list[n]))
print("train label: ", train_labels[n])
print("train before min: ", np.min(train_check), "train before max: ", np.max(train_check))
print("train after min: ", np.min(train[n]), "train after max: ", np.max(train[n]))
plt.subplot(1,2,1)
plt.imshow(train_check)
plt.subplot(1,2,2)
plt.imshow(train[n])
plt.show()

In [ ]:
#show processed image valid
n = 50
valid_check = cv2.imread(os.path.join(train_path, valid_list[n]))
print("valid label: ", valid_labels[n])
print("valid before min: ", np.min(valid_check), "valid before max: ", np.max(valid_check))
print("valid after min: ", np.min(valid[n]), "valid after max: ", np.max(valid[n]))
plt.subplot(1,2,1)
plt.imshow(valid_check)
plt.subplot(1,2,2)
plt.imshow(valid[n])
plt.show()

In [ ]:
#show processed image test
n = 50
test_check = cv2.imread(os.path.join(test_path, test_list[n]))
print("test before min: ", np.min(test_check), "test before max: ", np.max(test_check))
print("test after min: ", np.min(test[n]), "test after max: ", np.max(test[n]))
plt.subplot(1,2,1)
plt.imshow(test_check)
plt.subplot(1,2,2)
plt.imshow(test[n])
plt.show()

**Define Model**

In [ ]:
#copied
model = Sequential()

model.add(Convolution2D(64, 3, 3, activation='relu', border_mode='same', input_shape=(224, 224, 1)))
model.add(Convolution2D(64, 3, 3, activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Convolution2D(128, 3, 3, activation='relu', border_mode='same'))
model.add(Convolution2D(128, 3, 3, activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Convolution2D(256, 3, 3, activation='relu', border_mode='same'))
model.add(Convolution2D(256, 3, 3, activation='relu'))
model.add(Convolution2D(256, 3, 3, activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1))
model.add(Activation('sigmoid'))

model.compile(loss='categorical_crossentropy', optimizer='rmsprop', metrics=['accuracy'])

In [ ]:
# basic! v1

#always start with:
model = Sequential()

# add a 2D convolutional layer
# has 20 feature maps that are 50x50 and uses a rectifier (relu) activation function
# input shape defines the shape
model.add(Convolution2D(64, 3, 3, border_mode='same', input_shape=(img_size_row, img_size_col, 1), activation='relu'))

# next is a pooling layer (pools all the convolutions back together)
model.add(MaxPooling2D(pool_size=(2, 2)))

# next is the drop out layer
model.add(Dropout(0.2))

# next is the flattening which allows a full connected layer
model.add(Flatten())

# next is the dense fully connected layer with 128 neurons
model.add(Dense(128, activation='relu'))

# then activation
model.add(Activation('softmax'))

# then compile the beast
model.compile(loss='categorical_crossentropy', optimizer=RMSprop(lr=1e-4), metrics=['accuracy'])

In [ ]:
# run model on training set
train_labels_ = np.to_categorical(train_labels) #convert labels to a matrix representation
#train_labels_ = train_labels
train_ = np.resize(train, (len(train), img_size_row, img_size_col, 1)) # adding the extra dimension for no rgb

model.fit(train_, train_labels_, nb_epoch=1, batch_size=32)

In [ ]:
# run model on validate set
valid_labels_ = to_categorical(valid_labels, 2)
valid_ = np.resize(valid, (len(valid), img_size_row, img_size_col, 1))

print("train set :", model.evaluate(train_, train_labels_, verbose=False)[1]*100, "%")
print("--------------------")
print("valid set :", model.evaluate(valid_, valid_labels_, verbose=False)[1]*100, "%")

**Score out the final test set!!!**

In [ ]:
# run model on test set
test_ = np.resize(test, (len(test), img_size_row, img_size_col, 1))
model_out = model.predict([test_])

# save predictions
predictions = model_out[:,0] # only want the 1st column (index[0])

In [ ]:
print("test list length: ", len(test_list))
print()
print(test_list[:10])
print()
print("test score length: ", len(predictions))  
print()
print(predictions[:10])

In [ ]:
submission = pd.DataFrame({"id":test_list, "label": predictions})
submission.to_csv('submission.csv', index=False)

In [ ]:
for i in range(0,10):
    if predictions[i] >= 0.5: 
        print('I am {:.2%} sure this is a Dog'.format(predictions[i]))
    else: 
        print('I am {:.2%} sure this is a Cat'.format(1-predictions[i]))
                     
    plt.imshow(test[i])
    plt.show()